In [546]:
import pandas as pd
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import string
from tqdm import tqdm as tq
from collections import Counter
import math
import numpy as np
from heapq import *
#import geocoder
import heapq
import random
from itertools import chain 

In [547]:
##Setting seed to get deterministic hash Id's for vocabulary and inverted_index
PYTHONHASHSEED=12345

In [548]:
### Reading the dataset
data=pd.read_csv('Full_Dataset_Cleaned.csv')
data=data.drop('Unnamed: 0',axis=1)

data

,placeName,placeTags,numPeopleVisited,numPeopleWant,placeDesc,placeShortDesc,placeNearby,placeAddress,placeAlt,placeLong,placeEditors,placePubDate,placeRelatedLists,placeRelatedPlaces,placeURL
0,City Hall Station,"Manhattan, New York",1824,8569,The first New York City subway was built and o...,"City Hall Station,Manhattan, New York, A beaut...","['African Burial Ground National Monument', 'T...","31 Centre StNew York, New York, 10007United St...",40.7134,-74.0046,"['charding407', 'fosterc827', 'Mike H', 'erjef...","May 8, 2010",['30 Unexpected Places to Have a Joyful Advent...,"['Crystal Palace Subway', 'Moscow Metro Statio...",https://www.atlasobscura.com/places/city-hall-...
1,Highgate Cemetery,"London, England",2615,8179,"Opened in 1839, Highgate is one of London’s mo...","Highgate Cemetery,London, England, London's cr...","['Parkland Walk', ""World's Largest Potted Plan...","Swain's Lane, HighgateLondon, England, N6Unite...",51.5675,-0.1483,"['thesecretcity', 'thedyerghoulhouse', 'juliap...","August 9, 2014","[""The World's Top 100 Wonders in 2018"", ""Londo...","['Jewett City Vampires', 'Tomb of the Mather F...",https://www.atlasobscura.com/places/highgate-c...
2,Leadenhall Market,"London, England",3133,7560,The ornate 19th-century painted roof and cobbl...,"Leadenhall Market,London, England, This ornate...","[""London's Original and All-Inspiring Coffee H...","London, England, EC3VUnited Kingdom",51.5126,-0.0834,"['Julius Spada', 'Katecrimson', 'thesecretcity...","August 1, 2016","['The Ultimate Guide to Stunning, Surprising, ...","['Rivendell', 'Bagdad Cafe', 'Gare de la Ciota...",https://www.atlasobscura.com/places/leadenhall...
3,The Wave Organ,"San Francisco, California",2426,7400,"Located on a jetty in the San Francisco Bay, t...","The Wave Organ,San Francisco, California, A hu...","['Palace of Fine Arts', 'Long Now Orrery', 'Th...","83 Marina Green DrSan Francisco, California, 9...",37.8085,-122.4401,"['routt1111', 'wefiddleboy14', 'Saal333', 'Nei...","November 21, 2008","[""Leonardo Nam's 16 Quirky Roadside Attraction...","['Sea Organ', 'Silent Green Kulturquartier', ""...",https://www.atlasobscura.com/places/wave-organ
4,Catacombes de Paris,"Paris, France",4442,7036,"In 2004, Parisian police were assigned to do a...","Catacombes de Paris,Paris, France, The vast, l...","['Sculptures de Décure', 'Arago Medallions', ""...","1 Place Denfert-RochereauParis, 75014France",48.8343,2.3322,"['marypippen', 'ramonrodz2212', 'mathewbrowne'...","February 13, 2009","['19 Catacombs Sure to Tingle Your Spine', ""Th...","['Ossario di San Martino', 'Leuk Charnel House...",https://www.atlasobscura.com/places/catacombes...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7195,Xul Solar Museum,"Buenos Aires, Argentina",129,405,With dreams of reforming and perfecting the un...,"Xul Solar Museum,Buenos Aires, Argentina, Xul ...","['Ricardo Rojas House Museum', 'El Ateneo Gran...",Laprida 1212Buenos AiresArgentina,-34.5948,-58.4078,['Allison'],"September 12, 2016","[""A Linguistics Lover's Tour of the World""]","['Blenko Man', 'Ilana Goor Museum', 'Museo de ...",https://www.atlasobscura.com/places/xul-solar-...
7196,Klatovy Catacomb Mummies,"Klatovy, Czechia",49,405,In an underground crypt beneath a Jesuit churc...,"Klatovy Catacomb Mummies,Klatovy, Czechia, The...","['Poledník Lookout Tower', 'Entropa', 'Drachen...",DenisovaKlatovyCzechia,49.3951,13.2918,"['Max Cortesi ', ' hrnick ', ' marjolein']","February 25, 2016","['19 Catacombs Sure to Tingle Your Spine', ""Bo...","['Rothwell Bone Crypt', 'Ossario di San Martin...",https://www.atlasobscura.com/places/klatovy-ca...
7197,Termesphere Gallery,"Spearfish, South Dakota",71,405,"Most artists paint on a a flat plane, but in t...","Termesphere Gallery,Spearfish, South Dakota, T...","['Mount Roosevelt Friendship Tower', 'Mt. Mori...","1920 Christensen DriveSpearfish, South Dakota,...",44.4574,-103.8258,['kgsn143'],"August 19, 2014",NaN,"['Museo del Barro (Museum of Clay)', 'M

In [549]:
def preprocess(text:str):
    ### Converting to lowercase
    text = text.lower()
    
    ### Removing Punctuation
    text_p = "".join([char for char in text if char not in string.punctuation])
    
    ### Converting strings to tokens of individual words in a complete string
    words = word_tokenize(text_p)
    
    ### Remove Stopwords
    stop_words = stopwords.words('english')
    filtered_words = [word for word in words if word not in stop_words]
    
    ### Get the root of the word i.e stemming
    porter = PorterStemmer()
    stemmed = [porter.stem(word) for word in filtered_words]
    
    return stemmed

In [550]:
## For each document combining the three different columns and preprocessing the values to get all the required words
data_small=data[['placeDesc','placeAddress','placeName','placeEditors','placeTags','numPeopleVisited','placeRelatedLists']]
data_small['placeName_processed']=data_small['placeName'].apply(lambda row:preprocess(row))
data_small['placeAddress_processed']=data_small['placeAddress'].apply(lambda row:preprocess(row))
data_small['placeDesc_processed']=data_small['placeDesc'].apply(lambda row:preprocess(row))
data_small['placeTags']=data['placeTags']
data_small['combined_processed']=data_small['placeAddress'].apply(lambda row:preprocess(row))+data_small['placeName'].apply(lambda row:preprocess(row))+data_small['placeDesc'].apply(lambda row:preprocess(row))

C:\Users\paolo\AppData\Local\Temp\ipykernel_15356\2265716259.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_small['placeName_processed']=data_small['placeName'].apply(lambda row:preprocess(row))
C:\Users\paolo\AppData\Local\Temp\ipykernel_15356\2265716259.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_small['placeAddress_processed']=data_small['placeAddress'].apply(lambda row:preprocess(row))
C:\Users\paolo\AppData\Local\Temp\ipykernel_15356\2265716259.py:5: SettingWithCopyWarning: 
A va

In [551]:
def inverted_index(data_small):
    param=['placeAddress_processed','placeName_processed','placeDesc_processed']
    hist_voc={}
    hist_inverted_index={}
    document_id=[]
    for i in param:
        ### Combining all the words for all the document from those three fields and passing it 
        ### to counter object to get unique words and finally creating a dictionary of the word:hash_id
        word_list=list(chain.from_iterable(data_small[i].values)) ###Combine all the rows
        word_counter=Counter(word_list) ### Find unique words
        
        ### Create vocabulary dictionary of word(key) and it's id(value)
        vocabulary={key:hash(key) for key in word_counter.keys()} 
        vocabulary_df=pd.DataFrame.from_dict(vocabulary,orient='index',columns=['Term_id'])
        hist_voc[i]=vocabulary_df
        
        inverted_index={key:[] for key in vocabulary.keys()}
        
        for doc_num,j in tq(enumerate(data_small[i].values)):
            unique_j=list(set(j))
            for k in unique_j:
                inverted_index[k].append(dict({hash('document'+str(doc_num)):(j.count(k)/len(j))}))
            if i=='placeAddress_processed':
                document_id.append(hash('document'+str(doc_num)))
                
        ### For dataframe to assign a complete list to a single cell it has to be list(list(value))
        inverted_index={k:[v] for k,v in inverted_index.items()}
        inverted_index_df = pd.DataFrame.from_dict(inverted_index,orient='index',columns=['tf_id'])
        inverted_index_df['Term_id']=vocabulary_df['Term_id']
        inverted_index_df=inverted_index_df[['Term_id','tf_id']]
        hist_inverted_index[i]=inverted_index_df
        
    return hist_voc,hist_inverted_index,document_id

In [15]:
vocabulary,inverted_index_tf,document_id=inverted_index(data_small) 

7200it [00:00, 183602.80it/s]
7200it [00:00, 266653.03it/s]
7200it [00:02, 2927.39it/s]


### Adding generated Hash keys for Individual Documents to the dataset

In [552]:
data_small['Hash_id']=document_id
data_small

C:\Users\paolo\AppData\Local\Temp\ipykernel_15356\4023309973.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_small['Hash_id']=document_id


,placeDesc,placeAddress,placeName,placeEditors,placeTags,numPeopleVisited,placeRelatedLists,placeName_processed,placeAddress_processed,placeDesc_processed,combined_processed,Hash_id
0,The first New York City subway was built and o...,"31 Centre StNew York, New York, 10007United St...",City Hall Station,"['charding407', 'fosterc827', 'Mike H', 'erjef...","Manhattan, New York",1824,['30 Unexpected Places to Have a Joyful Advent...,"[citi, hall, station]","[31, centr, stnew, york, new, york, 10007unit,...","[first, new, york, citi, subway, built, oper, ...","[31, centr, stnew, york, new, york, 10007unit,...",4124632914409416820
1,"Opened in 1839, Highgate is one of London’s mo...","Swain's Lane, HighgateLondon, England, N6Unite...",Highgate Cemetery,"['thesecretcity', 'thedyerghoulhouse', 'juliap...","London, England",2615,"[""The World's Top 100 Wonders in 2018"", ""Londo...","[highgat, cemeteri]","[swain, lane, highgatelondon, england, n6unit,...","[open, 1839, highgat, one, london, ’, infam, c...","[swain, lane, highgatelondon, england, n6unit,...",-2926278338565634421
2,The ornate 19th-century painted roof and cobbl...,"London, England, EC3VUnited Kingdom",Leadenhall Market,"['Julius Spada', 'Katecrimson', 'thesecretcity...","London, England",3133,"['The Ultimate Guide to Stunning, Surprising, ...","[leadenhal, market]","[london, england, ec3vunit, kingdom]","[ornat, 19thcenturi, paint, roof, cobbl, floor...","[london, england, ec3vunit, kingdom, leadenhal...",-849568378175347152
3,"Located on a jetty in the San Francisco Bay, t...","83 Marina Green DrSan Francisco, California, 9...",The Wave Organ,"['routt1111', 'wefiddleboy14', 'Saal333', 'Nei...","San Francisco, California",2426,"[""Leonardo Nam's 16 Quirky Roadside Attraction...","[wave, organ]","[83, marina, green, drsan, francisco, californ...","[locat, jetti, san, francisco, bay, wave, orga...","[83, marina, green, drsan, francisco, californ...",-5969199895526255908
4,"In 2004, Parisian police were assigned to do a...","1 Place Denfert-RochereauParis, 75014France",Catacombes de Paris,"['marypippen', 'ramonrodz2212', 'mathewbrowne'...","Paris, France",4442,"['19 Catacombs Sure to Tingle Your Spine', ""Th...","[catacomb, de, pari]","[1, place, denfertrochereaupari, 75014franc]","[2004, parisian, polic, assign, train, exercis...","[1, place, denfertrochereaupari, 75014franc, c...",-4048817287110447579
...,...,...,...,...,...,...,...,...,...,...,...,...
7195,With dreams of reforming and perfecting the un...,Laprida 1212Buenos AiresArgentina,Xul Solar Museum,['Allison'],"Buenos Aires, Argentina",129,"[""A Linguistics Lover's Tour of the World""]","[xul, solar, museum]","[laprida, 1212bueno, airesargentina]","[dream, reform, perfect, univers, argentin, ar...","[laprida, 1212bueno, airesargentina, xul, sola...",2772799367162699114
7196,In an underground crypt beneath a Jesuit churc...,DenisovaKlatovyCzechia,Klatovy Catacomb Mummies,"['Max Cortesi ', ' hrnick ', ' marjolein']","Klatovy, Czechia",49,"['19 Catacombs Sure to Tingle Your Spine', ""Bo...","[klatovi, catacomb, mummi]",[denisovaklatovyczechia],"[underground, crypt, beneath, jesuit, church, ...","[denisovaklatovyczechia, klatovi, catacomb, mu...",5588621208426307541
7197,"Most artists paint on a a flat plane, but in t...","1920 Christensen DriveSpearfish, South Dakota,...",Termesphere Gallery,['kgsn143'],"Spearfish, South Dakota",71,NaN,"[termespher, galleri]","[1920, christensen, drivespearfish, south, dak...","[artist, paint, flat, plane, termespher, galle...","[1920, christensen, drivespearfish, south, dak...",-4751800887608136622
7198,At least six mountains in the southwestern Uni...,"Wheeler PeakTaos Ski Valley, New Mexico, 87525...",Wheeler Peak,['mikewalker'],"Taos Ski Valley, New Mexico",241,['The Highest Places in America'],"[wheeler, peak]","[wheeler, peaktao, ski, valley, new, mexico, 8...","[least, six, mountain, southwestern, unit, sta...","[wheeler, peaktao, ski, valley, new, mexico, 8...",85814961005574

In [57]:
### Exapmle of first among the three vocabulary DataFrame Created
vocabulary['placeDesc_processed']

,Term_id
first,-5780207401325983843
new,-6898394521187064627
york,-8779892923012318072
citi,-6377025136432249502
subway,5101848860868773510
...,...
13167,3591716107828038479
bullofthewood,-4607968237759408304
olana,-6241104219595475724
persianstyl,5061053960244213499


In [553]:
def get_tf_idf(row):
        for i in row:
            for k,v in i.items():
                i[k]=v*math.log(7200/len(row)) ### To change: Paramterise the number of total documents

In [554]:
#### Converting tf of the different terms for containing documents to tf_idf
inverted_index_tf_idf={}
for k,i in inverted_index_tf.items():
    print(k)
    var=i
    
#     var.reset_index(inplace=True)
    
    var[var.columns.values[-1]].apply(lambda row:get_tf_idf(row))
    inverted_index_tf_idf[k]=var

placeAddress_processed
placeName_processed
placeDesc_processed


In [555]:
inverted_index_tf_idf['placeDesc_processed']

,Term_id,tf_id
first,-5780207401325983843,"[{4124632914409416820: 0.011556634511120613}, ..."
new,-6898394521187064627,"[{4124632914409416820: 0.0304380491241755}, {1..."
york,-8779892923012318072,"[{4124632914409416820: 0.27101899522050565}, {..."
citi,-6377025136432249502,"[{4124632914409416820: 0.0682847692089397}, {-..."
subway,5101848860868773510,"[{4124632914409416820: 0.5456368088878675}, {-..."
...,...,...
13167,3591716107828038479,[{8581496100557481987: 5.88791230272821}]
bullofthewood,-4607968237759408304,[{8581496100557481987: 5.88791230272821}]
olana,-6241104219595475724,[{6053532931276860204: 14.69919365086693}]
persianstyl,5061053960244213499,[{6053532931276860204: 4.899731216955643}]


In [556]:
def get_document_id_tf_idf_vector(query,vocabulary_df,inverted_index_dataframe):
    
    # Main Objective:Get the Document_id's of the matched documents and it's tf_id representation vectors for all the documents
    
    query_p=preprocess(query)
    query_processed=[]
    for i in query_p:  ##### Remove duplicates
        if i not in query_processed:
            query_processed.append(i)
        
    ### Get the term_id from vocabulary
    term_id=[vocabulary_df[vocabulary_df.index==i].Term_id.values[0] for i in query_processed]

    ## Get the dictionary of {document_id:tf_idf scores} for each terms.
    query_documents=[inverted_index_dataframe[inverted_index_dataframe.Term_id==i].tf_id.values[0] for i in term_id]

    #### get the document_id for each of the term
    query_documents_keys=[]
    for i in range(len(query_documents)):
        list_query=[list(j.keys())[0] for j in query_documents[i]]
        query_documents_keys.append(list_query)

    ####Find the intersection of document_id's for all the terms to get those document_id which contains all the queries
    if len(query_documents_keys)>1:
        for i in range(1,len(query_documents_keys)):
            if i==1:
                document_id=set(query_documents_keys[i-1]).intersection(query_documents_keys[i])
            else:
                document_id=document_id.intersection(query_documents_keys[i])
    else:
        document_id=query_documents_keys[0]

    ### Based on the retrieved common document_id get the values for tf_idf score for each terms.
    query_documents_values=[]
    for i in range(len(query_documents)):
        list_query=[list(i.values())[0] for i in query_documents[i] if list(i.keys())[0] in document_id]
        query_documents_values.append(list_query)

    ### Create document vector whose individual length corresponds the number of queries.
    query_document=zip(*query_documents_values)
    query_document_vector=[i for i in query_document]
    
    return list(document_id),query_document_vector

In [557]:
def get_query_vector(query,vocabulary_df,inverted_index_dataframe,data_des):
    
    ## Pre-processing query
    query_processed=preprocess(query)
    
    ### Get the term_id from vocabulary
    term_id=[vocabulary_df[vocabulary_df.index==i].Term_id.values[0] for i in query_processed]


    ## Get the documents index from inverted index     ###### check if there are more occorrences of term[i]                                                                       
    query_idf=[np.log(len(data_des)/len(inverted_index_dataframe[inverted_index_dataframe.Term_id==term_id[i]].tf_id.values[0])) for i in range(len(term_id)) if term_id[i] not in term_id[:i]]
    query_tf=[query_processed.count(query_processed[i])/len(query_processed) for i in range(len(query_processed)) if query_processed[i] not in query_processed[:i]]

    query_vector=np.multiply(query_idf,query_tf)
    return query_vector

In [558]:
def calculate_cosine_similarity(query_vector:list,query_document_vector):
    
    z=np.dot(query_vector,query_document_vector)/(np.linalg.norm(query_vector)*np.linalg.norm(query_document_vector))

# z=z.sum()/math.sqrt(len(query_processed)) ### Take care of the parameterisation of the calculation,it would lenght of the quer

    z=z.sum()        
    return z

In [559]:
def Search_engine_top_k(query,vocabulary_df,inverted_index_dataframe,data_des,k):

        ### Get document id common to all queries and it's tf_id representation
        document_id,query_document_vector=get_document_id_tf_idf_vector(query,vocabulary_df,inverted_index_dataframe)
        
        ### Get Query Vector
        query_vector=get_query_vector(query,vocabulary_df,inverted_index_dataframe,data_des)
        
        ### Scoring Metric defined for this choice
        metric=[]
        
        #### Converting list to heap
        heapq.heapify(metric)
        
    
        ### Variable for top_k documents
        max_length=k
        
        #### Iterate over the document id's and get similarity score & Position Score for each document to get the final score
        for i,j in enumerate(document_id):
                    
                    
                     ### Cosine Similarity based on tf_idf representation of the document
                     score=calculate_cosine_similarity(query_vector,query_document_vector[i])
                  
                     ### Storing upto the size of the heap
                     if i<max_length:
                        heappush(metric,(score,j))

                     ## Replacing the minumum element from the heap(Gaol is to maintain the top scores)
                     else:
                        if score>metric[0][0]:
                            heapreplace(metric,(score,j))
                            
        ### Get the sorted order of the metric: Each element is (Score,Document_id)        
        metric=heapq.nlargest(k,metric)
        
        ### Get the Final DataFrame
        for i,j in enumerate(metric):
                if i==0:
                    final_df=data_des[data_des['Hash_id']==j[1]]
                    final_df=final_df.assign(Score=j[0])
                else:
                    x=data_des[data_des['Hash_id']==j[1]]
                    x=x.assign(Score=j[0])
                    final_df=pd.concat([final_df,x])          
        return final_df#.drop(['processed','hash_key','placeNearby','numPeopleWant'],axis=1)

In [560]:
query='american history'
Search_engine_top_k(query,vocabulary['placeDesc_processed'],inverted_index_tf_idf['placeDesc_processed'],data_small,244)

,placeDesc,placeAddress,placeName,placeEditors,placeTags,numPeopleVisited,placeRelatedLists,placeName_processed,placeAddress_processed,placeDesc_processed,combined_processed,Hash_id,Score
7064,There is only one place in Nevada where you wi...,"1595 N Sierra StReno, NevadaUnited States",Wilbur D. May Museum,"['Molly McBride Jacobson ', ' saramarvelous']","Reno, Nevada",131,NaN,"[wilbur, may, museum]","[1595, n, sierra, streno, nevadaunit, state]","[one, place, nevada, find, shrunken, human, he...","[1595, n, sierra, streno, nevadaunit, state, w...",3920321101383390739,0.998988
5324,"Known as “Vatnasafn” in the native Icelandic, ...",Bókhlöðustígur 17StykkishólmurIceland,The Library of Water,"['meaganw ', ' mwhite']","Stykkishólmur, Iceland",82,NaN,"[librari, water]","[bókhlöðustígur, 17stykkishólmuriceland]","[known, “, vatnasafn, ”, nativ, iceland, libra...","[bókhlöðustígur, 17stykkishólmuriceland, libra...",9005526068499060983,0.998988
2603,Motoring down the scenic drive west on Interst...,"Golden, ColoradoUnited States",Sculptured House,"['brandybatz', 'maevekreilly', 'kcwatson8', 'b...","Golden, Colorado",607,['50 Strangely Made Houses In The United States'],"[sculptur, hous]","[golden, coloradounit, state]","[motor, scenic, drive, west, interst, 70, away...","[golden, coloradounit, state, sculptur, hous, ...",8449903344147495866,0.998988
3917,The popularity of Asian-fusion food has soared...,"899 Burrard StreetVancouver, British Columbia,...",Japadog,"['SamR ', ' notoriousFIG ', ' wythe']","Vancouver, British Columbia",740,NaN,[japadog],"[899, burrard, streetvancouv, british, columbi...","[popular, asianfus, food, soar, last, decad, s...","[899, burrard, streetvancouv, british, columbi...",8217013011404681109,0.998988
173,Las Vegas is an oasis in the desert where no o...,"755 E Flamingo RdLas Vegas, Nevada, 89119Unite...",National Atomic Testing Museum,"['Taterchips', 'paigeelizabeth3660', 'dlc31723...","Las Vegas, Nevada",1093,['17 Hidden Las Vegas Spots It Would Be a Sin ...,"[nation, atom, test, museum]","[755, e, flamingo, rdla, vega, nevada, 89119un...","[la, vega, oasi, desert, one, sleep, everi, vi...","[755, e, flamingo, rdla, vega, nevada, 89119un...",5207256425929225703,0.998988
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4496,The Atlantic coast of South Carolina is home t...,"70 Honey Horn DriveHilton Head Island, South C...",Gullah Heritage Trail,"['leslipeterson ', ' seanjobst ', ' naturedude']","Hilton Head Island, South Carolina",181,"[""A Linguistics Lover's Tour of the World""]","[gullah, heritag, trail]","[70, honey, horn, drivehilton, head, island, s...","[atlant, coast, south, carolina, home, cultur,...","[70, honey, horn, drivehilton, head, island, s...",-3384178414600646682,0.906001
6202,The Siriraj Medical Museum abounds with medica...,"Siriraj Sub-District, Bangkoknoi DistrictBangk...",Siriraj Medical Museum,"['mbison ', ' dqcris ', ' Annetta Black ', ' v...","Bangkok, Thailand",246,NaN,"[siriraj, medic, museum]","[siriraj, subdistrict, bangkoknoi, districtban...","[siriraj, medic, museum, abound, medic, curios...","[siriraj, subdistrict, bangkoknoi, districtban...",-3921815051640999663,0.906001
2491,"Built in 1936, the U-Drop Inn in Shamrock, Tex...","1242 N Main StShamrock, TexasUnited States",U-Drop Inn,"['The Coppers', 'cait7911', 'breaingram', 'Ker...","Shamrock, Texas",378,NaN,"[udrop, inn]","[1242, n, main, stshamrock, texasunit, state]","[built, 1936, udrop, inn, shamrock, texa, kind...","[1242, n, main, stshamrock, texasunit, state, ...",-4636156300042179100,0.906001
2352,Property developer Michael Bauman was in for a...,"401 Brickell AvenueMiami, Florida, 33131United...",Miami Circle,"['ReubenHull47 ', ' rugby007 ', ' bill 15aba2a...","Miami, Florida",310,['39 Places That Will Warp Your Perspective of...,"[miami, circl]","[401, brickel, avenuemiami, florida, 33131unit...","[properti, develop, michael, bauman, real, sur...","[401, brickel, avenuemiami, florida, 33131unit...",-5

In [561]:
query='Name:museum,Add: ,Des:american museum -f1:Aty Trocious;Nemish ,f2:Denver,f3U:4000,f3l:1000,f4:17 Places to Hunt for Aliens Besides Area 51'

def query_process(query):
    # divide query in two part [Name:xxx, Des:xxx, Add:xxx], [f1:xxx,..., f4:xxx]
    query=query.split('-')

    query1=query[0]
    filter1=query[1]
    
    # Trasform the two strings in two lists of dicts 
    query1=query1.split(',')
    query1=[dict({i.split(':')[0]:i.split(':')[1]}) for i in query1 if i.split(':')[1] not in ['',' ']]
    filter1=filter1.split(',')
    filter1=[dict({i.split(':')[0]:i.split(':')[1].split(';')}) for i in filter1]
    return query1,filter1
query_process(query)

([{'Name': 'museum'}, {'Des': 'american museum '}],
 [{'f1': ['Aty Trocious', 'Nemish ']},
  {'f2': ['Denver']},
  {'f3U': ['4000']},
  {'f3l': ['1000']},
  {'f4': ['17 Places to Hunt for Aliens Besides Area 51']}])

In [562]:
query='Name:museum,Add:atlanta,Des:american museum -f1:Aty Trocious;SquiddyGiggles,f2:Denver,f3U:4000,f3l:1000,f4:17 Places to Hunt for Aliens Besides Area 51'
query,filt= query_process(query)

query,filt

([{'Name': 'museum'}, {'Add': 'atlanta'}, {'Des': 'american museum '}],
 [{'f1': ['Aty Trocious', 'SquiddyGiggles']},
  {'f2': ['Denver']},
  {'f3U': ['4000']},
  {'f3l': ['1000']},
  {'f4': ['17 Places to Hunt for Aliens Besides Area 51']}])

In [563]:
### filter_ find all the doc that contein the list of filter-query
def filter_(row,list1):
   x=0
   for i in list1: ### check counting the elements conteined in the row
     try:
        if i in row:
            x+=1
       
     except:
        continue
   if x==len(list1):
        
       return True
   else:
       return False

In [617]:
### nuovo
def filt(fil,data):
    
    try:
        ### filter the dataset using the filter_ function
        
        if list(fil[0].values())[0] not in [[' '],['']]:
            data = data[data['placeEditors'].apply(lambda row: filter_(row,list(fil[0].values())[0]))]
            
        if list(fil[1].values())[0] not in [[' '],['']]:
            data = data[data['placeTags'].apply(lambda row: filter_(row,list(fil[1].values())[0]))]
            
        if list(fil[2].values())[0] not in [[' '],['']]:
            data = data[data['numPeopleVisited']<=int(list(fil[2].values())[0][0])]
            
        if list(fil[3].values())[0] not in [[' '],['']]:
            data = data[data['numPeopleVisited']>=int(list(fil[3].values())[0][0])]
            
        if list(fil[4].values())[0] not in [[' '],['']]:
            data = data[data['placeRelatedLists'].apply(lambda row: filter_(row,list(fil[4].values())[0]))]
            
        return data
    
    except:
        print("There isn't documents that respect your query")
        return None 
            

In [622]:
def Search_engine_top_k_par(query,vocabulary_df,inverted_index_dataframe,data_des,k):
        
    param_dict={'Name':'placeName_processed','Add':'placeAddress_processed','Des':'placeDesc_processed'}
    query,fil=query_process(query)
    keys=[list(i.keys())[0] for i in query]

    df=pd.DataFrame()

    for i in query:
        par=param_dict[list(i.keys())[0]]
        part_query=list(i.values())[0]

        ### Get document id common to all queries and it's tf_id representation
        document_id,query_document_vector=get_document_id_tf_idf_vector(part_query,vocabulary_df[par],inverted_index_dataframe[par])
      
        ### Get Query Vector
        query_vector=get_query_vector(part_query,vocabulary_df[par],inverted_index_dataframe[par],data_des)
        
        ### Scoring Metric defined for this choice
        metric=[]

        #### Converting list to heap
        heapq.heapify(metric)

        ### Variable for top_k documents
        max_length = k

        #### Iterate over the document id's and get similarity score & Position Score for each document to get the final score
        for i,j in enumerate(document_id):


                     ### Cosine Similarity based on tf_idf representation of the document
                     score=calculate_cosine_similarity(query_vector,query_document_vector[i])

                     ### Storing upto the size of the heap
                     if i<max_length:
                        heappush(metric,(score,j))

                    ## Replacing the minumum element from the heap(Gaol is to maintain the top scores)
                     else:
                        if score>metric[0][0]:
                            heapreplace(metric,(score,j))

        ### Get the sorted order of the metric: Each element is (Score,Document_id)        
        metric=heapq.nlargest(k,metric)

        ### Get the Final DataFrame
        for i,j in enumerate(metric):
            if i==0:
                final_df=data_des[data_des.Hash_id.isin([j[1]])]
                final_df=final_df.assign(Score=j[0])
            else:
                x=data_des[data_des.Hash_id.isin([j[1]])]
                x=x.assign(Score=j[0])
                final_df=pd.concat([final_df,x])
             
        if len(query)!=1:
            final_df.rename(columns={'Score':'Score'+par.split('_')[0]},inplace=True)
        ### Concate the previous dataset with the new search
        df=pd.concat([df,final_df], axis=0)
        
        
    ### If the user have inserted some filter-query, use filt function to filter the dataset
    if fil!=[{'f1': ['']}, {'f2': ['']}, {'f3U': [' ']}, {'f3l': [' ']}, {'f4': ['']}]:
            try:
                df = filt(fil,df)
            except:
                print("There isn't documents that respect your query")
                return None
            
    if len(df)==0:
            print("There isn't documents that respect your query")
            return None  
        
    df=df.fillna(0)
    
    ### Give differents weight to the scores depending on the query inserted
    if len(query)!=1:
        if 'Name' not in keys:
            df['Score']=0.3*np.array(df['ScoreplaceDesc'])+0.7*np.array(df['ScoreplaceAddress'])
            df=df.drop(['ScoreplaceDesc','ScoreplaceAddress'], axis=1)
        elif 'Des' not in keys:
            df['Score']=0.6*np.array(df['ScoreplaceName'])+0.4*np.array(df['ScoreplaceAddress'])
            df=df.drop(['ScoreplaceName','ScoreplaceAddress'], axis=1)
        elif 'Add' not in keys:
            df['Score']=0.7*np.array(df['ScoreplaceName'])+0.3*np.array(df['ScoreplaceDesc'])
            df=df.drop(['ScoreplaceDesc','ScoreplaceAddress'], axis=1)
        else:
            df['Score']=0.2*np.array(df['ScoreplaceDesc'])+0.4*np.array(df['ScoreplaceAddress'])+0.4*np.array(df['ScoreplaceName'])
            df=df.drop(['ScoreplaceDesc','ScoreplaceAddress','ScoreplaceName'], axis=1)

    
    
    
    return df #.drop(['processed','hash_key','placeNearby','numPeopleVisited','numPeopleWant','placeAlt','placeLong'],axis=1) 


In [623]:
#query='Name:museum american,Add:atlanta,Des:american museum -f1:Aty Trocious;SquiddyGiggles,f2: ,f3U:4000,f3l:1000,f4:17 Places to Hunt for Aliens Besides Area 51'
query='Name:Cafebrería ,Add:alejandro ,Des:first -f1:linkogecko,f2:,f3U:500 ,f3l:200 ,f4:'

x=Search_engine_top_k_par(query,vocabulary,inverted_index_tf_idf,data_small,1000)

x

,placeDesc,placeAddress,placeName,placeEditors,placeTags,numPeopleVisited,placeRelatedLists,placeName_processed,placeAddress_processed,placeDesc_processed,combined_processed,Hash_id,Score
1062,This amazing chain of Mexico City bookstores n...,"Alejandro Dumas 81Mexico City, 11550Mexico",Cafebrería El Péndulo,"['linkogecko', 'maxmaya86', 'acoumbis', 'Monsi...","Mexico City, Mexico",462,['25 Coffee Shops to Visit Before You Die'],"[cafebrería, el, péndulo]","[alejandro, duma, 81mexico, citi, 11550mexico]","[amaz, chain, mexico, citi, bookstor, normal, ...","[alejandro, duma, 81mexico, citi, 11550mexico,...",9107045642048165636,0.4
1062,This amazing chain of Mexico City bookstores n...,"Alejandro Dumas 81Mexico City, 11550Mexico",Cafebrería El Péndulo,"['linkogecko', 'maxmaya86', 'acoumbis', 'Monsi...","Mexico City, Mexico",462,['25 Coffee Shops to Visit Before You Die'],"[cafebrería, el, péndulo]","[alejandro, duma, 81mexico, citi, 11550mexico]","[amaz, chain, mexico, citi, bookstor, normal, ...","[alejandro, duma, 81mexico, citi, 11550mexico,...",9107045642048165636,0.4
1062,This amazing chain of Mexico City bookstores n...,"Alejandro Dumas 81Mexico City, 11550Mexico",Cafebrería El Péndulo,"['linkogecko', 'maxmaya86', 'acoumbis', 'Monsi...","Mexico City, Mexico",462,['25 Coffee Shops to Visit Before You Die'],"[cafebrería, el, péndulo]","[alejandro, duma, 81mexico, citi, 11550mexico]","[amaz, chain, mexico, citi, bookstor, normal, ...","[alejandro, duma, 81mexico, citi, 11550mexico,...",9107045642048165636,0.2
3025,"Unknown to many, this passage is a short-cut b...","Jacob's LadderCalton RoadEdinburgh, Scotland, ...",Jacob's Ladder,"['linkogecko ', ' reley250']","Edinburgh, Scotland",439,0,"[jacob, ladder]","[jacob, laddercalton, roadedinburgh, scotland,...","[unknown, mani, passag, shortcut, edinburgh, ’...","[jacob, laddercalton, roadedinburgh, scotland,...",8333464072102280864,0.2
2315,While Frida Kahlo’s Casa Azul in nearby Coyoac...,Av. Altavista & Calle Diego RiveraMexico CityM...,Diego Rivera and Frida Kahlo House Studio Museum,"['Mario Yair TS ', ' linkogecko ', ' jesuisrob...","Mexico City, Mexico",413,0,"[diego, rivera, frida, kahlo, hous, studio, mu...","[av, altavista, call, diego, riveramexico, cit...","[frida, kahlo, ’, casa, azul, nearbi, coyoacan...","[av, altavista, call, diego, riveramexico, cit...",4035654306133180494,0.2
441,Those who think that Hector Guimard was the on...,"29 Avenue RappParis, 75007France",Jules Lavirotte's 29 Avenue Rapp,"['linkogecko', 'reley250', 'Blindcolour', 'SEA...","Paris, France",476,['33 Places Famous for the Scandals That Occur...,"[jule, lavirott, 29, avenu, rapp]","[29, avenu, rapppari, 75007franc]","[think, hector, guimard, eccentr, architect, p...","[29, avenu, rapppari, 75007franc, jule, laviro...",2124635883476720722,0.2
1094,"In 1864, the famous French world traveler Loui...","Shekhpura RoadAbhaneri, 303326India",Chand Baori,"['JatinChhabra ', ' linkogecko ', ' kostas1964...","Abhaneri, India",343,['Favorites'],"[chand, baori]","[shekhpura, roadabhaneri, 303326india]","[1864, famou, french, world, travel, loui, rou...","[shekhpura, roadabhaneri, 303326india, chand, ...",-399914191462765903,0.2
1658,The first question people tend to have when th...,1100 South 2nd Street2nd Street at Washington ...,Mummers Museum,"['rachelwjek', 'linkogecko', 'Brady Santoro', ...","Philadelphia, Pennsylvania",374,0,"[mummer, museum]","[1100, south, 2nd, street2nd, street, washingt...","[first, question, peopl, tend, hear, mummer, m...","[1100, south, 2nd, street2nd, street, washingt...",-903926438964407681,0.2
1734,"In 1651, Cromwell’s armies descended on the an...","Tantallon CastleNorth Berwick, Scotland, EH39 ...",Ruins of Tantallon Castle,"['linkogecko ', ' SEANETTA ', ' Martin ', ' Ra...","North Berwick, Scotland",269,0,"[ruin, tantallon, castl]","[tantallon, castlenorth, berwick, scotland, eh...","[1651, cromwel, ’, armi, descend, ancient, str...","[tantallon, castlenorth, berwick, sc